# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='./Reacher_Linux_1/Reacher.x86_64')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
import torch
from collections import deque

# train the agent
def train_ddpg(agent, memory, n_episodes=10, mode='train', 
        actor_pth='./checkpoint/ddpg_actor_checkpoint.pth',
        critic_pth='./checkpoint/ddpg_critic_checkpoint.pth'):
    
    '''Set up trainings configuration and print out episodic performance measures, such as average scores, average loss.
    
    Params
    ======
        agent (class object)
        memory (class attribute): agent's attribute for memory size tracking
        mode (string): 'train' or 'test', when in test mode, the agent acts in greedy policy only
        pth (string path): file name for the checkpoint    
    '''
    
    scores = []
    scores_window = deque(maxlen=100)  
    c_loss_window = deque(maxlen=100)
    a_loss_window = deque(maxlen=100)

    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment and activate train_mode
        state = env_info.vector_observations[0]            # get the current state
        score = 0
        agent.running_c_loss = 0
        agent.running_a_loss = 0
        agent.training_cnt = 0
        # agent.reset() # reset OUNoise
        
        while True:
            action = agent.act(state, mode)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished        

            agent.step(state, action, reward, next_state, done)

            score += reward
            state = next_state
            if done:
                break

        scores_window.append(score)
        scores.append(score)
        c_loss_window.append(agent.running_c_loss/(agent.training_cnt+0.0001)) # avoid zero
        a_loss_window.append(agent.running_a_loss/(agent.training_cnt+0.0001)) # avoid zero
        print('\rEpisode {:>4}\tAverage Score:{:>6.3f}\tMemory Size:{:>5}\tCLoss:{:>12.8f}\tALoss:{:>10.6f}'.format(
            i_episode, np.mean(scores_window), len(memory), np.mean(c_loss_window), np.mean(a_loss_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {:>4}\tAverage Score:{:>6.3f}\tMemory Size:{:>5}\tCLoss:{:>12.8f}\tALoss:{:>10.6f}'.format(
                i_episode, np.mean(scores_window), len(memory), np.mean(c_loss_window), np.mean(a_loss_window)))
        if np.mean(scores_window) >= 31:
            break
    torch.save(agent.actor_local.state_dict(), actor_pth)
    torch.save(agent.critic_local.state_dict(), critic_pth)
    return scores

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

def plot_scores(scores, rolling_window=100):
    '''Plot score and its moving average on the same chart.'''
    
    fig = plt.figure(figsize=(10,5))
    plt.plot(np.arange(len(scores)), scores, '-c', label='episode score')
    plt.title('Episodic Score')
    rolling_mean = pd.Series(scores).rolling(rolling_window).mean()
    plt.plot(np.arange(len(scores)), rolling_mean, '-y', label='rolling_mean')
    plt.ylabel('score')
    plt.xlabel('episode #')
    plt.legend()

In [ ]:
from tools import active_session
from ddpg_agent import Agent
agent_ddpg = Agent(state_size=33, action_size=4, seed=199)

ddpg_ep_rewards_list = train_ddpg(agent=agent_ddpg, memory=agent_ddpg.memory,
                                  n_episodes=1000)
ddpg_ep_rewards_list = np.array(ddpg_ep_rewards_list)
np.save('./data/ddpg_ep_rewards_list.npy', ddpg_ep_rewards_list)

In [ ]:
def train_d4pg(agent, memory, n_episodes=10, mode='train', 
        actor_pth='./checkpoint/d4pg_actor_checkpoint.pth',
        critic_pth='./checkpoint/d4pg_critic_checkpoint.pth'):
    
    '''Set up training configuration and print out episodic performance measures, such as average scores, average loss.
    
    Params
    ======
        agent (class object)
        memory (class attribute): agent's attribute for memory size tracking
        mode (string): 'train' or 'test', when in test mode, the agent acts in greedy policy only
        pth (string path): file name for the checkpoint    
    '''
    
    scores = []
    scores_window = deque(maxlen=100)  
    c_loss_window = deque(maxlen=100)
    a_loss_window = deque(maxlen=100)

    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment and activate train_mode
        state = env_info.vector_observations               # get the current state
        score = 0
        agent.running_c_loss = 0
        agent.running_a_loss = 0
        agent.training_cnt = 0
        # agent.reset() # reset OUNoise
        
        while True:
            action = agent.act(state, mode)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations      # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished        

            agent.step(state, action, reward, next_state, done)

            score += reward
            state = next_state
            if done:
                break

        scores_window.append(score)
        scores.append(score)
        c_loss_window.append(agent.running_c_loss/(agent.training_cnt+0.0001)) # avoid zero
        a_loss_window.append(agent.running_a_loss/(agent.training_cnt+0.0001)) # avoid zero
        print('\rEpisode {:>4}\tAverage Score:{:>6.3f}\tMemory Size:{:>5}\tCLoss:{:>12.8f}\tALoss:{:>10.6f}'.format(
            i_episode, np.mean(scores_window), len(memory), np.mean(c_loss_window), np.mean(a_loss_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {:>4}\tAverage Score:{:>6.3f}\tMemory Size:{:>5}\tCLoss:{:>12.8f}\tALoss:{:>10.6f}'.format(
                i_episode, np.mean(scores_window), len(memory), np.mean(c_loss_window), np.mean(a_loss_window)))
        if np.mean(scores_window) >= 31:
            break
    torch.save(agent.actor_local.state_dict(), actor_pth)
    torch.save(agent.critic_local.state_dict(), critic_pth)
    return scores

In [ ]:
from d4pg_agent import AgentD4PG
agent_d4pg = AgentD4PG(state_size=33, action_size=4, seed=168)

In [ ]:
d4pg_ep_rewards_list = train_d4pg(agent=agent_d4pg, memory=agent_d4pg.memory,
                                n_episodes=20000,
                                mode = "train",
                                actor_pth='./checkpoint/d4pg_actor_checkpoint.pth',
                                critic_pth='./checkpoint/d4pg_critic_checkpoint.pth')
d4pg_ep_rewards_list = np.array(d4pg_ep_rewards_list)
np.save('./data/d4pg_ep_rewards_list.npy', d4pg_ep_rewards_list)

In [ ]:
plot_scores(d4pg_ep_rewards_list)

In [ ]:
env.close()

# Training Multiple Agents

In [ ]:
env = UnityEnvironment(file_name='./Reacher_Linux_20/Reacher.x86_64')

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

In [ ]:
from a2cModel import A2CModel
from a2cModel import collect_trajectories, learn

In [ ]:
#from IPython.core.debugger import set_trace
#set_trace()

from collections import deque

torch.backends.cudnn.deterministic = True
torch.manual_seed(999)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

agent_a2c = A2CModel().to(device)
optimizer = optim.Adam(agent_a2c.parameters(), lr=0.00015)

env_info = env.reset(train_mode=True)[brain_name] 
states = env_info.vector_observations
init_states = states

n_episodes = 1
n_steps = 10
episode_end = False
a2c_ep_rewards_list = []
ep_rewards_deque = deque([0], maxlen=100) # initialize with 0
ep_rewards = 0

while True:
    batch_s, batch_a, batch_v_t, accu_rewards, init_states, episode_end = collect_trajectories(
            agent_a2c, env, brain_name, init_states, episode_end, n_steps)

    loss, mus, stds = learn(batch_s, batch_a, batch_v_t, agent_a2c, optimizer)
    ep_rewards += accu_rewards
    print('\rEpisode {:>4}\tEpisodic Score {:>7.3f}\tLoss {:>12.6f}'.format(
            n_episodes, np.mean(ep_rewards_deque), float(loss)), end="")

    if episode_end == True:
        if n_episodes % 100 == 0:
            print('\rEpisode {:>4}\tEpisodic Score {:>7.3f}\tLoss {:>12.6f}'.format(
                    n_episodes, np.mean(ep_rewards_deque), float(loss)))

        if np.mean(ep_rewards_deque) >= 34:
            break
        a2c_ep_rewards_list.append(ep_rewards/num_agents)
        ep_rewards_deque.append(ep_rewards/num_agents)
        ep_rewards = 0
        n_episodes += 1
        episode_end = False
            
# save a2c model
pth = './checkpoint/a2c_checkpoint.pth'
torch.save(agent_a2c.state_dict(), pth)

a2c_ep_rewards_list = np.array(a2c_ep_rewards_list)
np.save('./data/a2c_ep_rewards_list.npy', a2c_ep_rewards_list)

In [ ]:
plot_scores(a2c_ep_rewards_list)

In [ ]:
# Visualization of all models

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

ddpg_ep_rewards_list = np.load('./data/ddpg_ep_rewards_list.npy')
a2c_ep_rewards_list = np.load('./data/a2c_ep_rewards_list.npy')
d4pg_ep_rewards_list = np.load('./data/d4pg_ep_rewards_list.npy')

ddpg_ep_rewards_list_rolling_mean = pd.Series(ddpg_ep_rewards_list).rolling(100).mean()
a2c_ep_rewards_list_rolling_mean = pd.Series(a2c_ep_rewards_list).rolling(100).mean()
d4pg_ep_rewards_list_rolling_mean = pd.Series(d4pg_ep_rewards_list).rolling(100).mean()

# plot all models
fig = plt.figure(figsize=(10,5))
plt.title('Compare rolling means of the rewards of the employed algorithms')
plt.plot(np.arange(len(a2c_ep_rewards_list_rolling_mean)), a2c_ep_rewards_list_rolling_mean, '-g', label='a2c')
plt.plot(np.arange(len(d4pg_ep_rewards_list_rolling_mean)), d4pg_ep_rewards_list_rolling_mean, '-y', label='d4pg')
plt.plot(np.arange(len(ddpg_ep_rewards_list_rolling_mean)), ddpg_ep_rewards_list_rolling_mean, '-r', label='ddpg')
plt.ylabel('rolling mean')
plt.xlabel('episode #')
plt.legend()
plt.show()